In [ ]:
# !pip install google-cloud-aiplatform[prediction]>=1.16.0 fastapi nvtabular git+https://github.com/NVIDIA-Merlin/models.git --user

# Buidling a custom Vertex AI endpoint for Merlin Query Tower

**IMPORTANT** Make sure you are running this notebook in a DLVM (e.g. tensorflow enterprise 2.8) to build the image

________
**This will not work in the training container**
________

Your output should look like this - you are going to use the query model endpoint to create a custom container

![](img/merlin-bucket.png)


In [1]:
# !gsutil cp -r $workflow_uri local_workflow

In [2]:
# !gsutil cp -r gs://spotify-beam-v3/merlin-processed/workflow/query-tower-model-22-12 local_model

#### END DEBUG

In [3]:
from datetime import datetime


PROJECT = 'wortz-project-352116'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1'
REGION = 'us-central1'
# path = 'gs://two-tower-models' #TODO change to your model directory
BUCKET = 'gs://spotify-jsw-mpd-2023'
REPOSITORY = 'merlin-spotify-cpr'
ARTIFACT_URI = f'{BUCKET}'
MODEL_DIR = f'{BUCKET}/query_model_merlin'
PREFIX = 'merlin-spotify'

In [4]:
# run one time to locally copy the workflow for the image

In [5]:
# !mkdir workflow
# !gsutil cp -r $BUCKET/workflow workflow

In [6]:
# !gsutil mb -l us-central1 gs://wortz-project-bucket

In [7]:
# !echo y | docker container prune
# !echo y | docker image prune

### Set up repo and configure Docker (one-time)

In [8]:
# Create the repo if needed for the artifacts

! gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [9]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [10]:
! rm -rf app
! mkdir app

### Dependency file
The first few are for the server handling traffic
Nvtabular was downgraded for this example, it may not be necessary in future versions

In [11]:
%%writefile app/requirements.txt
uvicorn[standard]==0.15.0
gunicorn==20.1.0
fastapi==0.68.1
google-cloud-aiplatform
merlin-models
nvtabular
gcsfs
google-cloud-storage

Writing app/requirements.txt


### Predictor module and class
This is an adaptation of the CPR examples

This was locally tested and created shortly after model creation in the [tensorflow-predict](tensorflow-predict.ipynb) notebook

In [12]:
%%writefile app/dataset_to_tensors.py

try:
    import cudf
except ImportError:
    cudf = None
import pandas as pd
import tensorflow as tf
from typing import Dict
from merlin.io import Dataset
import itertools


def cupy_array_to_tensor(array):
    return tf.experimental.dlpack.from_dlpack(array.reshape(-1, 1).toDlpack())

def numpy_array_to_tensor(array):
    return tf.convert_to_tensor(array.reshape(-1, 1))

def cudf_series_to_tensor(col) -> tf.Tensor:
    "Convert a cudf.Series to a TensorFlow Tensor with DLPack"
    if isinstance(col.dtype, cudf.ListDtype):
        values = col.list.leaves.values
        offsets = col.list._column.offsets.values
        row_lengths = offsets[1:] - offsets[:-1]
        return cupy_array_to_tensor(values), cupy_array_to_tensor(row_lengths)
    else:
        return cupy_array_to_tensor(col.values)

def pandas_series_to_tensor(col) -> tf.Tensor:
    if len(col) and pd.api.types.is_list_like(col.values[0]):
        values = pd.Series(itertools.chain(*col)).values
        row_lengths = col.map(len).values
        return numpy_array_to_tensor(values), numpy_array_to_tensor(row_lengths)
    else:
        return numpy_array_to_tensor(col.values)
        
    
def dataset_to_tensors(dataset: Dataset) -> Dict[str, tf.Tensor]:
    """Convert a DataFrame to Dict of Tensors"""
    df = dataset.to_ddf().compute()
    if isinstance(df, pd.DataFrame):
        col_to_tensor = pandas_series_to_tensor
    else:
        col_to_tensor = cudf_series_to_tensor
    return {
        column: col_to_tensor(df[column])
        for column in df.columns
    }

Writing app/dataset_to_tensors.py


In [13]:
%%writefile app/predictor.py

import nvtabular as nvt
import pandas as pd
import os
import json
import merlin.models.tf as mm
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()
import tensorflow as tf
import logging
from dataset_to_tensors import *
import time


# These are helper functions that ensure the dictionary input is in a certain order and types are preserved
# this is to get scalar values to appear first in the dict to not confuse pandas with lists https://github.com/pandas-dev/pandas/issues/46092
reordered_keys = ['collaborative', 'album_name_pl', 'artist_genres_pl', 
                  'artist_name_pl', 'artist_pop_can', 'description_pl', 
                  'duration_ms_songs_pl', 'n_songs_pl', 'name', 'num_albums_pl', 
                  'num_artists_pl', 'track_name_pl', 'track_pop_pl', 
                  'duration_ms_seed_pl', 'pid', 'track_uri_pl']  

float_num_fix = ['n_songs_pl','num_albums_pl','num_artists_pl','duration_ms_seed_pl']
float_list_fix = ['track_pop_pl', 'duration_ms_songs_pl']
    
def fix_list_num_dtypes(num_list):
    "this fixes lists of ints to list of floats converted in json input"
    return [float(x) for x in num_list]

def fix_num_dtypes(num):
    "this fixes ints and casts to floats"
    return float(num)

def fix_types(k, v):
    if k in float_num_fix:
        return fix_num_dtypes(v)
    if k in float_list_fix:
        return fix_list_num_dtypes(v)
    else:
        return v

def create_pandas_instance(inputs):
    """
    Helper function to reorder the input to have a sclar first for pandas
    And fix the types converted when data is imported by fastAPI
    """
    if type(inputs) == list:
        header = inputs[0]
        reordered_header_dict = {k: fix_types(k,header[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_header_dict, orient='index').T
        if len(inputs) > 1:
            for ti in inputs[1:]:
                reordered_dict = {k: fix_types(k,ti[k]) for k in reordered_keys}
                pandas_instance = pandas_instance.append(pd.DataFrame.from_dict(reordered_dict, orient='index').T)
    else:
        reordered_dict = {k: fix_types(k,inputs[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_dict, orient='index').T
    return pandas_instance

class Predictor():
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """
    def __init__(self):
        return
    
    def load(self, artifacts_uri):
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        logging.info("loading model and workflow")
        start_init = time.process_time()
        
        #test_bucket = 'gs://jt-merlin-scaling'
        self.model = tf.keras.models.load_model(f"{artifacts_uri}/query_model_merlin")
        # self.workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow")) # TODO: parameterize
        self.workflow = nvt.Workflow.load("/docker_workflow/2t-spotify-workflow")
        # self.workflow = nvt.Workflow.load('gs://jt-merlin-scaling/nvt-last5-v1full/nvt-analyzed') # TODO: parametrize
        self.workflow = self.workflow.remove_inputs(
            [
                'track_pop_can', 
                'track_uri_can', 
                'duration_ms_can', 
                'track_name_can', 
                'artist_name_can',
                'album_name_can',
                'album_uri_can',
                'artist_followers_can', 
                'artist_genres_can',
                'artist_name_can', 
                'artist_pop_can',
                'artist_pop_pl',
                'artist_uri_can', 
                'artists_followers_pl'
            ]
        )
        return self
        
    def predict(self, prediction_input):
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        # handle different input types, can take a dict or list of dicts
        self.n_rows = len(prediction_input)
        start = time.process_time()
        pandas_instance = create_pandas_instance(prediction_input[0])
        #logging.info(f"Pandas conversion took {time.process_time() - start} seconds")
        print(f"Pandas conversion took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_inputs = nvt.Dataset(pandas_instance)
        #logging.info(f"NVT data loading took {time.process_time() - start} seconds")
        print(f"NVT data loading took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_instance = self.workflow.transform(transformed_inputs)
        print(f"Workflow transformation took {time.process_time() - start} seconds")

        # def predict(self, instances):
        start = time.process_time()
        
        batch = dataset_to_tensors(transformed_instance)
        print(f"converting to dict_tensors took {time.process_time() - start} seconds")
        start = time.process_time()
        output = self.model(batch)
        print(f"Generating query embeddings took {time.process_time() - start} seconds")
        return transformed_instance, output, batch

Writing app/predictor.py


In [14]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import json
import numpy as np
import os
import logging


from google.cloud import storage
from predictor import Predictor

app = FastAPI()

predictor_instance = Predictor()
loaded_predictor = predictor_instance.load(artifacts_uri = os.environ['AIP_STORAGE_URI'])

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    outputs = loaded_predictor.predict(instances)

    return {"predictions": outputs[1].numpy().tolist()}

Writing app/main.py


In [15]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing app/prestart.sh


In [16]:
%%writefile app/instances.json
{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_can": "We Just Havent Met Yet", "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_can": 57.0, "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_can": "spotify:track:0VzDv4wiuZsLsNOmfaUy2W", "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}}

Writing app/instances.json


In [17]:
#make it a package
!touch app/__init__.py

In [18]:
%%writefile Dockerfile

FROM nvcr.io/nvidia/merlin/merlin-tensorflow:23.02
WORKDIR /app 

COPY ./app/requirements.txt /requirements.txt
RUN pip install -r /requirements.txt
#DEBUG CHANGES!!
RUN mkdir /docker_workflow
# RUN mkdir /docker_model
# ADD local_model /docker_model
ADD workflow /docker_workflow
#END DEBUG!


COPY ./app /app
EXPOSE 80
    
CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

Overwriting Dockerfile


In [19]:
SERVER_IMAGE = "merlin-prediction-cpr"  # @param {type:"string"} 
REMOTE_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{SERVER_IMAGE}"

!docker build -t $REMOTE_IMAGE_NAME .
# !gcloud builds submit -t $REMOTE_IMAGE_NAME .

Sending build context to Docker daemon  39.73GB
Step 1/9 : FROM nvcr.io/nvidia/merlin/merlin-tensorflow:23.02
 ---> 7f446a9c1777
Step 2/9 : WORKDIR /app
 ---> Running in f309967bed21
Removing intermediate container f309967bed21
 ---> c52121bfbe68
Step 3/9 : COPY ./app/requirements.txt /requirements.txt
 ---> 61eb04deb584
Step 4/9 : RUN pip install -r /requirements.txt
 ---> Running in d8344678f9b1
ERROR: dask-cudf 22.8.0a0+304.g6ca81bbc78.dirty requires cupy-cuda118<12,>=9.5.0, which is not installed.
ERROR: cudf 22.8.0a0+304.g6ca81bbc78.dirty requires cupy-cuda118<12,>=9.5.0, which is not installed.
ERROR: tensorflow 2.10.1+nv22.12 has requirement flatbuffers>=2.0, but you'll have flatbuffers 1.12 which is incompatible.
ERROR: tensorflow 2.10.1+nv22.12 has requirement tensorboard<2.11,>=2.10, but you'll have tensorboard 2.9.1 which is incompatible.
ERROR: tensorflow 2.10.1+nv22.12 has requirement tensorflow-estimator<2.11,>=2.10.0, but you'll have tensorflow-estimator 2.9.0 which is i

#### If you are debugging, be sure to set `-d` detached flag off and run the commands in console

us-central1-docker.pkg.dev/wortz-project-352116/merlin-spotify-cpr/merlin-prediction-cp### Copy/paste if you want to run from console for testing
```python
docker run --gpus all -p 80:8080 \
            --name=merlin-prediction-cpr \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-jsw-mpd-2023 \
            us-central1-docker.pkg.dev/wortz-project-352116/merlin-spotify-cpr/merlin-prediction-cpr
```

##### No GPU:
```python
docker run -p 80:8080 \
            --name=merlin-prediction-cpr \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-jsw-mpd-2023 \
            us-central1-docker.pkg.dev/wortz-project-352116/merlin-spotify-cpr/merlin-prediction-cpr
```

#### Test the health route

In [61]:
! curl localhost/health

{}

In [62]:
## Ground truth candidate:
    # 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
    # 'artist_name_can': 'Russ', 
    # 'track_name_can': 'We Just Havent Met Yet', 
## TODO - we have to overload with candidate data because of the workflow transform, add overloaded values in the predictor
TEST_INSTANCE = {'collaborative': 'false',
                 'album_name_pl': ["There's Really A Wolf", 'Late Nights: The Album',
                       'American Teen', 'Crazy In Love', 'Pony'], 
                 'artist_genres_pl': ["'hawaiian hip hop', 'rap'",
                       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
                       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
                       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], 
                 'artist_name_pl': ['Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9',
                       'William Singe'], 
                 'artist_pop_can': 82.0, 
                 'description_pl': '', 
                 'duration_ms_songs_pl': [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], 
                 'n_songs_pl': 8.0, 
                 'name': 'Lit Tunes ', 
                 'num_albums_pl': 8.0, 
                 'num_artists_pl': 8.0, 
                 'track_name_pl': ['Losin Control', 'Paradise', 'Location',
                       'Crazy In Love - Remix', 'Pony'], 
                 'track_pop_pl': [79.0, 58.0, 83.0, 71.0, 57.0],
                 'duration_ms_seed_pl': 51023.1,
                 'pid': 1,
                 'track_uri_pl': ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
                       'spotify:track:1wNEBPo3nsbGCZRryI832I',
                       'spotify:track:152lZdxL1OR0ZMW6KquMif',
                       'spotify:track:2f4IuijXLxYOeBncS60GUD',
                       'spotify:track:4Lj8paMFwyKTGfILLELVxt']
                     }

In [63]:
import json
import requests

json_instance = json.dumps({"instances": [TEST_INSTANCE]})
print(json_instance)

{"instances": [{"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "description_pl": "", "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "sp

### Test the predict route

In [66]:
%%time
requests.post('http://localhost/predict', data=json_instance).json()

CPU times: user 5.13 ms, sys: 153 µs, total: 5.28 ms
Wall time: 535 ms


{'predictions': [[0.3048475682735443,
   0.0,
   1.2088731527328491,
   0.05392572283744812,
   0.0,
   0.27251535654067993,
   0.9764062166213989,
   0.4936431646347046,
   0.0,
   0.0,
   0.5621760487556458,
   0.0,
   0.569015383720398,
   0.1336423009634018,
   0.0,
   0.0,
   0.0,
   0.0,
   0.9370920062065125,
   1.1970816850662231,
   0.35134124755859375,
   0.0,
   0.19488777220249176,
   1.8816297054290771,
   0.05862294137477875,
   0.0,
   0.86396723985672,
   0.31114858388900757,
   0.0,
   0.18993006646633148,
   0.0,
   0.00311265978962183,
   0.1541297435760498,
   0.6337419152259827,
   0.2538793087005615,
   0.0,
   0.7494783401489258,
   0.0,
   1.2598177194595337,
   0.023529257625341415,
   0.0,
   0.008459048345685005,
   0.18115344643592834,
   0.6338210105895996,
   0.0,
   0.09700655937194824,
   0.008082836866378784,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.20535233616828918,
   0.0,
   0.19821757078170776,
   0.0,
   0.9934800863265991,
   0.0,
   0.0,
   

## Stop the images if they are running

In [67]:
! docker stop $SERVER_IMAGE
! docker rm $SERVER_IMAGE

merlin-prediction-cpr
merlin-prediction-cpr


### Push the container once ready and testing is complete

In [68]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr]

d3c4334f: Preparing 
e72fcee7: Preparing 
b1721332: Preparing 
22aee77e: Preparing 
0d58e7c2: Preparing 
6878a4fe: Preparing 
2e747ad7: Preparing 
46caf956: Preparing 
c4f4a731: Preparing 
6f8ac904: Preparing 
55970ccc: Preparing 
1e82e910: Preparing 
7c23e98f: Preparing 
70ded861: Preparing 
da794091: Preparing 
493f5d44: Preparing 
caa9de82: Preparing 
03ad2a31: Preparing 
5e1b9e48: Preparing 
f4addbfc: Preparing 
81e7ae87: Preparing 
3d5ddcb4: Preparing 
a992b719: Preparing 
f10e020a: Preparing 
7f70fff9: Preparing 
2562fda1: Preparing 
decbb6f6: Preparing 
1cea2b76: Preparing 
fcaf77a9: Preparing 
e31ca3c7: Preparing 
f58ddb35: Preparing 
61786022: Preparing 
1f7b3d75: Preparing 
e5c17b0b: Preparing 
15f32335: Preparing 
fb54d10a: Preparing 
90be1950: Preparing 
84fa095a: Preparing 
cab8959e: Preparing 
7d764727: Preparing 
d4d05916: Preparing

### Deploy to Vertex AI

After the serving metadata is set below, the model is properly abstracted for use on Vertex

In [ ]:
MODEL_DISPLAY_NAME = "Merlin Spotify Query Tower Model - Workflow Local"
from google.cloud import aiplatform

#DEBUG FOR HARDCODING
ARTIFACT_URI = 'gs://spotify-beam-v3/merlin-processed/workflow/query-tower-model-22-12'

model = aiplatform.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=ARTIFACT_URI,
        serving_container_image_uri=REMOTE_IMAGE_NAME,
        serving_container_predict_route='/predict',
        serving_container_health_route='/health',
        serving_container_command=["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"],
        serving_container_args='--gpus all',
        sync=True,
    )

Creating Model
Create Model backing LRO: projects/934903580331/locations/us-central1/models/1394482010205978624/operations/2804597770088873984


In [ ]:
endpoint = model.deploy(machine_type="n1-standard-8",
                        accelerator_type="NVIDIA_TESLA_T4",
                        accelerator_count=1)

In [72]:
#make a prediction

endpoint.predict(instances=[[TEST_INSTANCE, TEST_INSTANCE]])

Prediction(predictions=[[0.3048475384712219, 0.0, 1.20887303352356, 0.05392575263977051, 0.0, 0.2725152969360352, 0.9764063954353333, 0.493643045425415, 0.0, 0.0, 0.5621761679649353, 0.0, 0.5690153241157532, 0.1336423009634018, 0.0, 0.0, 0.0, 0.0, 0.9370918273925781, 1.197081804275513, 0.3513412773609161, 0.0, 0.1948878020048141, 1.881629586219788, 0.05862289667129517, 0.0, 0.86396723985672, 0.3111485242843628, 0.0, 0.1899300664663315, 0.0, 0.003112689591944218, 0.1541297435760498, 0.6337419748306274, 0.2538793385028839, 0.0, 0.749478280544281, 0.0, 1.259817719459534, 0.02352931722998619, 0.0, 0.00845898874104023, 0.1811534762382507, 0.6338210105895996, 0.0, 0.09700654447078705, 0.008082814514636993, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2053523659706116, 0.0, 0.1982176005840302, 0.0, 0.9934801459312439, 0.0, 0.0, 0.9782760143280029, 0.3004560768604279, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.30164635181427, 0.0, 0.02676838636398315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7641364932060242, 3.025083541870117, 0.

## Finished - now go on to the next notebook to create a [matching engine notebook](03-matching-engine.ipynb) and test out the first end to end recommendation

Be sure to use the output of the endpoint logs above to save the endpoint for use in the matching engine notebook

e.g.:

```python
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/934903580331/locations/us-central1/endpoints/494907775848022016')
```

### Timing analysis

In [73]:
%%timeit
endpoint.predict(instances=[[TEST_INSTANCE, TEST_INSTANCE]])

601 ms ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
%%timeit
endpoint.predict(instances=[[TEST_INSTANCE]])

598 ms ± 17.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
endpoint.undeploy_all()

Undeploying Endpoint model: projects/934903580331/locations/us-central1/endpoints/4736674102226452480
Undeploy Endpoint model backing LRO: projects/934903580331/locations/us-central1/endpoints/4736674102226452480/operations/6606761755496415232
Endpoint model undeployed. Resource name: projects/934903580331/locations/us-central1/endpoints/4736674102226452480


resource name: projects/934903580331/locations/us-central1/endpoints/4736674102226452480